<a href="https://colab.research.google.com/github/mlKwon/Multi_Modal_Fusion_ERC/blob/main/try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp39-cp39-linux_x86_64.whl size=680533 sha256=55cd054dae4def55350531e8e33c865844edef26be5f90d311348eb605b4f83b
  Stored in directory: /root/.cache/pip/wheels/47/17/70/b257bc53879a458c4bfcc900e89271aa8b4f19366a54bd2455
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

In [ ]:
import os
import glob

os.chdir("drive/MyDrive/2023aifactory/wav")
dirpath = os.path.join(os.getcwd(),os.listdir()[0])
audio_path = glob.glob(dirpath + "/*.txt")

audio_path

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW # 인공지능 모델의 초기값 지정 함수를 아담으로 지정한다.
from transformers.optimization import get_cosine_schedule_with_warmup

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

model, vocab = get_pytorch_kobert_model()

# from kobert_tokenizer import KoBERTTokenizer
# tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
torch.cuda.device_count()
if torch.cuda.is_available():    
    device = torch.device("cuda")

tokenizer = get_tokenizer()
vocab[0]

In [ ]:
import pandas as pd
import re
annot = pd.read_csv('/content/drive/MyDrive/2023aifactory/annotation/Sess01_eval.csv')
id = annot.iloc[1:,3].tolist()
emotion = annot.iloc[1:,4]
emotion[emotion.str.contains(";")] = emotion[emotion.str.contains(";")].str.split(";").str[0]
emotion = emotion.tolist()
valence = annot.iloc[1:,5].tolist()
arousal = annot.iloc[1:,6].tolist()


In [ ]:
dic_ = {'happy':0,'surprise':1,'angry':2,'neutral':3,'disqust':4,'fear':5,'sad':6}

def removeTag(text):
  text = text.replace("c/","")
  text = text.replace("n/","")
  text = text.replace("N/","")
  text = text.replace("u/","")
  text = text.replace("l/","")
  text = text.replace("b/","")
  text = text.replace("*","")
  text = text.replace("+","")
  text = text.replace("/","")
  text = text.replace("\n","")
  text = text.replace("/","  ").replace("/","  ").replace("/","  ").lstrip().rstrip()
  return text

In [ ]:
lst = [['',0] for i in range(64) ]

for i in range(64):  
  # dat=nlp.data.TSVDataset(audio_path[0], encoding='cp949')
  f = open(os.getcwd() + "/Session01/"+id[i]+".txt",'r', encoding='cp949')
  line = f.readline()
  lst[i][0] = removeTag(line)
  lst[i][1] = dic_[emotion[i]]
  # print(lst)
  f.close()

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(lst, 0, 1, tok, max_len, True, False)
# data_train[0]

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)

using cached model. /content/drive/MyDrive/2023aifactory/wav/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    

In [ ]:
for e in range(num_epochs):
  train_acc = 0.0
  model.train()
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
      optimizer.zero_grad()
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)
      loss = loss_fn(out, label)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
      optimizer.step()
      scheduler.step()  # Update learning rate schedule
      train_acc += calc_accuracy(out, label)
      if batch_id % log_interval == 0:
          print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
  print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

<ipython-input-91-42f14fad32cd>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 1.477081060409546 train acc 0.625
epoch 1 train acc 0.625


  0%|          | 0/1 [00:00<?, ?it/s]

...

epoch 3 batch id 1 loss 1.4181745052337646 train acc 0.71875
epoch 3 train acc 0.71875


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 1.387174367904663 train acc 0.734375
epoch 4 train acc 0.734375


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 1.4090607166290283 train acc 0.734375
epoch 5 train acc 0.734375


In [ ]:
model(token_ids, valid_length, segment_ids)[0,:]

tensor([ 0.3171, -0.1569, -0.4962,  0.6436, -0.3868, -0.3005,  0.0401],
       device='cuda:0', grad_fn=<SliceBackward0>)